In [39]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
from extract.polocm import AP, Hypothesis, POLOCM
from traces import *
from convertor import TopoConvertor
from observation import PartialOrderedActionObservation, ObservedPartialOrderTraceList
from extract.model import *
import pulp as pl
from tabulate import tabulate

In [41]:
objects = {
    "A": PlanningObject("unknown", "A"),
    "B": PlanningObject("unknown", "B"),
    "C": PlanningObject("unknown", "C"),
}

actions = {
    "unstackAB": Action("unstack", [objects["A"], objects["B"]]),
    "unstackBC": Action("unstack", [objects["B"], objects["C"]]),
    "unstackCB": Action("unstack", [objects["C"], objects["B"]]),
    "unstackBA": Action("unstack", [objects["B"], objects["A"]]),
    "unstackAC": Action("unstack", [objects["A"], objects["C"]]),
    "unstackCA": Action("unstack", [objects["C"], objects["A"]]),
    "stackAB": Action("stack", [objects["A"], objects["B"]]),
    "stackBA": Action("stack", [objects["B"], objects["A"]]),
    "stackAC": Action("stack", [objects["A"], objects["C"]]),
    "stackCA": Action("stack", [objects["C"], objects["A"]]),
    "stackBC": Action("stack", [objects["B"], objects["C"]]),
    "stackCB": Action("stack", [objects["C"], objects["B"]]),
    "putdownA": Action("putdown", [objects["A"]]),
    "putdownB": Action("putdown", [objects["B"]]),
    "putdownC": Action("putdown", [objects["C"]]),
    "pickupA": Action("pickup", [objects["A"]]),
    "pickupB": Action("pickup", [objects["B"]]),
    "pickupC": Action("pickup", [objects["C"]]),
}

traces = TraceList(
    [
        Trace(
            [
                Step(State({}), actions["unstackAB"], 0),
                Step(State({}), actions["putdownA"], 1),
                Step(State({}), actions["pickupB"], 2),
                Step(State({}), actions["stackBC"], 3),
                Step(State({}), actions["unstackBC"], 4),
                Step(State({}), actions["stackBC"], 5),
                Step(State({}), actions["pickupA"], 6),
                Step(State({}), actions["stackAB"], 7),
            ]
        ),
        Trace(
            [
                Step(State({}),actions['unstackCB'], 0),
                Step(State({}),actions['putdownC'], 1),
                Step(State({}),actions['unstackBA'], 2),
                Step(State({}),actions['putdownB'], 3),
                Step(State({}),actions['pickupB'], 4),
                Step(State({}),actions['stackBC'], 5),
                Step(State({}),actions['pickupA'], 6),
                Step(State({}),actions['stackAB'], 7),

            ]
        ),
        Trace(
            [
                Step(State({}), actions["unstackCB"], 0),
                Step(State({}), actions["stackCA"], 1),
                Step(State({}), actions["pickupB"], 2),
                Step(State({}), actions["putdownB"], 3),
                Step(State({}), actions["pickupB"], 4),
                Step(State({}), actions["stackBC"], 5),
            ]
        ),
        
    ]
)


In [42]:
convertor = TopoConvertor("flex", True, True, 2)
po_tracelist = traces.topo(convertor, 0.3)
print(tabulate(po_tracelist[0].comparable_matrix, headers='keys', tablefmt="fancy_grid"))
obs_tracelist = po_tracelist.tokenize(PartialOrderedActionObservation, ObservedPartialOrderTraceList)

╒════════════╤══════════════╤══════════════╤═════════════╤════════════╤══════════════╤════════════╤═════════════╤════════════╕
│            │   unstack[0] │   putdown[1] │   pickup[2] │   stack[3] │   unstack[4] │   stack[5] │   pickup[6] │   stack[7] │
╞════════════╪══════════════╪══════════════╪═════════════╪════════════╪══════════════╪════════════╪═════════════╪════════════╡
│ unstack[0] │          nan │            1 │           1 │        nan │            1 │          1 │           1 │          1 │
├────────────┼──────────────┼──────────────┼─────────────┼────────────┼──────────────┼────────────┼─────────────┼────────────┤
│ putdown[1] │          nan │          nan │           1 │        nan │            1 │          1 │           1 │          1 │
├────────────┼──────────────┼──────────────┼─────────────┼────────────┼──────────────┼────────────┼─────────────┼────────────┤
│ pickup[2]  │          nan │          nan │         nan │        nan │          nan │          1 │         nan

In [47]:
debug = {
    'pstep6': True,
    '2step0': True
}
model = POLOCM(obs_tracelist, debug=debug)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/uto/miniconda3/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/0073bbacd7274f91aa273b7ff4585358-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/0073bbacd7274f91aa273b7ff4585358-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 2287 COLUMNS
At line 8448 RHS
At line 10731 BOUNDS
At line 11235 ENDATA
Problem MODEL has 2282 rows, 503 columns and 5118 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 10.8333 - 0.02 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 501 strengthened rows, 62 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 204 strengthened rows, 10 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 62 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 19 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightene

## Solution FO matrix

### Trace 0

#### Obj zero

╒════════════╤══════════════╤════════════╤══════════════╤═════════════╤═════════════╤════════════╤══════════════╤════════════╕
│            │   [1]putdown │   [7]stack │   [0]unstack │   [2]pickup │   [6]pickup │   [3]stack │   [4]unstack │   [5]stack │
╞════════════╪══════════════╪════════════╪══════════════╪═════════════╪═════════════╪════════════╪══════════════╪════════════╡
│ [1]putdown │          nan │          0 │            0 │           1 │           0 │          0 │            0 │          0 │
├────────────┼──────────────┼────────────┼──────────────┼─────────────┼─────────────┼────────────┼──────────────┼────────────┤
│ [7]stack   │            0 │        nan │            0 │           0 │           0 │          0 │            0 │          0 │
├────────────┼──────────────┼────────────┼──────────────┼─────────────┼─────────────┼────────────┼──────────────┼────────────┤
│ [0]unstack │            1 │          0 │          nan │           0 │           0 │          0 │            0

...

Checking candidate set *{putdown.1, stack.2, unstack.2}* of **Sort.2** for well formedness and Validity

,putdown.1,stack.2,unstack.2
putdown.1,0,1,0
stack.2,0,0,1
unstack.2,0,1,0


0 1
0 2
1 2
This subset is well-formed.
This subset is valid.
Adding this subset {putdown.1, stack.2, unstack.2} to the locm2 transition set.
Hole that is covered now:
[putdown.1, unstack.2]
[{unstack.1, putdown.1}, {unstack.1, stack.2, unstack.2}, {stack.1, unstack.2, pickup.1}, {putdown.1, stack.2, pickup.1}, {putdown.1, stack.2, unstack.2}]

Removed redundancy transition set list
[{unstack.1, putdown.1}, {unstack.1, stack.2, unstack.2}, {stack.1, unstack.2, pickup.1}, {putdown.1, stack.2, pickup.1}, {putdown.1, stack.2, unstack.2}]


#### Final transition set list

[{unstack.1, putdown.1}, {unstack.1, stack.2, unstack.2}, {stack.1, unstack.2, pickup.1}, {putdown.1, stack.2, pickup.1}, {putdown.1, stack.2, unstack.2}, {pickup.1, unstack.2, stack.1, unstack.1, putdown.1, stack.2}]
TS_overall: 
 [{FSM.0 @ Sort.0: {zero zero: [unstack.0, putdown.0, pickup.0, stack.0, unstack.0, stack.0, pickup.0, stack.0]}, FSM.0 @ Sort.1: {unknown A: [unstack.1, putdown.1], unknown B: [unstack.1], unknown C: []}, FSM.1 @ Sort.1: {unknown A: [unstack.1], unknown B: [unstack.2, unstack.1, stack.2], unknown C: [stack.2, unstack.2, stack.2]}, FSM.2 @ Sort.1: {unknown A: [pickup.1, stack.1], unknown B: [unstack.2, pickup.1, stack.1, stack.1], unknown C: [unstack.2]}, FSM.3 @ Sort.1: {unknown A: [putdown.1, pickup.1], unknown B: [pickup.1, stack.2], unknown C: [stack.2, stack.2]}, FSM.4 @ Sort.1: {unknown A: [putdown.1], unknown B: [unstack.2, stack.2], unknown C: [stack.2, unstack.2, stack.2]}, FSM.5 @ Sort.1: {unknown A: [unstack.1, putdown.1, pickup.1, stack.1], unknow

In [46]:
model.to_pddl('polocm')

putdown
{(fsmFSM.5 @ Sort.1_state1 sort1), (fsmFSM.4 @ Sort.1_state0 sort1), (fsmFSM.0 @ Sort.1_state1 sort1), (fsmFSM.3 @ Sort.1_state0 sort1)}
[x0 (sort1)]
(fsmFSM.5 @ Sort.1_state1 sort1)
[0]
(fsmFSM.4 @ Sort.1_state0 sort1)
[0]
(fsmFSM.0 @ Sort.1_state1 sort1)
[0]
(fsmFSM.3 @ Sort.1_state0 sort1)
[0]
pickup
{(fsmFSM.5 @ Sort.1_state1 sort1), (fsmFSM.2 @ Sort.1_state3 sort1), (fsmFSM.3 @ Sort.1_state2 sort1)}
[x0 (sort1)]
(fsmFSM.5 @ Sort.1_state1 sort1)
[0]
(fsmFSM.2 @ Sort.1_state3 sort1)
[0]
(fsmFSM.3 @ Sort.1_state2 sort1)
[0]
unstack
{(fsmFSM.2 @ Sort.1_state2 sort1), (fsmFSM.1 @ Sort.1_state2 sort1 sort1), (fsmFSM.4 @ Sort.1_state2 sort1 sort1), (fsmFSM.5 @ Sort.1_state0 sort1 sort1), (fsmFSM.5 @ Sort.1_state1 sort1), (fsmFSM.0 @ Sort.1_state0 sort1), (fsmFSM.1 @ Sort.1_state1 sort1 sort1)}
[x0 (sort1), x1 (sort1)]
(fsmFSM.2 @ Sort.1_state2 sort1)
[1]
(fsmFSM.1 @ Sort.1_state2 sort1 sort1)
[1, 0]
(fsmFSM.4 @ Sort.1_state2 sort1 sort1)
[1, 0]
(fsmFSM.5 @ Sort.1_state0 sort1 sor